In [1]:
import pandas as pd
from collections import defaultdict
import math
import numpy as np
import string

In [2]:
punct = set(string.punctuation)

noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
verb_suffix = ["ate", "ify", "ise", "ize"]
adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
adv_suffix = ["ward", "wards", "wise"]


def get_word_tag(line, vocab): 
    if not line.split():
        word = "--n--"
        tag = "--s--"
        return word, tag
    else:
        word, tag = line.split()
        if word not in vocab: 
            word = assign_unk(word)
        return word, tag
    return None 


def preprocess(vocab, data_fp):
    orig = []
    prep = []

    with open(data_fp, "r") as data_file:

        for cnt, word in enumerate(data_file):
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word)
                prep.append(word)
                continue
            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep


def assign_unk(tok):
    if any(char.isdigit() for char in tok):
        return "--unk_digit--"
    elif any(char in punct for char in tok):
        return "--unk_punct--"
    elif any(char.isupper() for char in tok):
        return "--unk_upper--"
    elif any(tok.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"
    elif any(tok.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"
    elif any(tok.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"
    elif any(tok.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"
    return "--unk--"


In [3]:
with open("WSJ-2_21.pos.txt", 'r') as f:
    training_corpus = f.readlines()

with open("hmm_vocab.txt", 'r') as f:
    voc_l = f.read().split('\n')

vocab = {} 
for i, word in enumerate(sorted(voc_l)): 
    vocab[word] = i       
    

cnt = 0
for k,v in vocab.items():
    cnt += 1
    if cnt > 20:
        break

with open("WSJ-24.pos.txt", 'r') as f:
    y = f.readlines()

_, prep = preprocess(vocab, "test.words.txt")

In [4]:
def create_dictionaries(training_corpus, vocab):
    
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    prev_tag = '--s--' 

    i = 0 

    for word_tag in training_corpus:

        i += 1

        if i % 50000 == 0:
            print(f"word count = {i}")
        word,tag = get_word_tag(word_tag,vocab)
        transition_counts[(prev_tag,tag)] += 1
        emission_counts[(tag,word)] += 1
        tag_counts[tag] += 1
        prev_tag = tag
        
    return emission_counts, transition_counts, tag_counts

In [7]:
emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)

word count = 50000
word count = 100000
word count = 150000
word count = 200000
word count = 250000
word count = 300000
word count = 350000
word count = 400000
word count = 450000
word count = 500000
word count = 550000
word count = 600000
word count = 650000
word count = 700000
word count = 750000
word count = 800000
word count = 850000
word count = 900000
word count = 950000


In [10]:
states = sorted(tag_counts.keys())
print(f"Number of POS tags (number of 'states'): {len(states)}")
print("View these POS tags (states)")
print(states)

Number of POS tags (number of 'states'): 46
View these POS tags (states)
['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [11]:
def predict_pos(prep, y, emission_counts, vocab, states):

    correct = 0

    words_tuple = set(emission_counts.keys())

    total_words = len(y)
    for word, y_tup in zip(prep, y):

        y_tup_list = y_tup.split()

        if len(y_tup_list) == 2:

            trueposlabel = y_tup_list[1]

        else:

            continue
    
        total_count = 0
        total_pos = ''

        if word in vocab:
            for pos in states:
                key = (pos, word)
                if key in emission_counts:
                    count = emission_counts[key]
                    if count > total_count:
                        total_count = count
                        total_pos = pos
            if total_pos == trueposlabel:
                correct += 1

    accuracy = correct / total_words
    
    return accuracy

In [12]:
accuracy_predict_pos = predict_pos(prep, y, emission_counts, vocab, states)
print(f"Accuracy of prediction using predict_pos is {accuracy_predict_pos:.4f}")

Accuracy of prediction using predict_pos is 0.8889


In [13]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
    tags = sorted(tag_counts.keys())
    num_tags = len(tags)
    A = np.zeros((num_tags,num_tags))
    
    for i in range(num_tags):
        for j in range(num_tags):
            counter = 0
            k = (tags[i],tags[j])
            if k in transition_counts:
                counter = transition_counts[k]

            count_previous = tag_counts[k[0]]
            A[i,j] = (counter + alpha)/(count_previous + num_tags*alpha)
    
    return A

In [14]:
alpha = 0.001
A = create_transition_matrix(alpha, tag_counts, transition_counts)
print(f"A at row 0, col 0: {A[0,0]:.9f}")
print(f"A at row 3, col 1: {A[3,1]:.4f}")

print("View a subset of transition matrix A")
A_sub = pd.DataFrame(A[30:35,30:35], index=states[30:35], columns = states[30:35] )
print(A_sub)

A at row 0, col 0: 0.000007040
A at row 3, col 1: 0.1691
View a subset of transition matrix A
              RBS            RP           SYM        TO            UH
RBS  2.217069e-06  2.217069e-06  2.217069e-06  0.008870  2.217069e-06
RP   3.756509e-07  7.516775e-04  3.756509e-07  0.051089  3.756509e-07
SYM  1.722772e-05  1.722772e-05  1.722772e-05  0.000017  1.722772e-05
TO   4.477336e-05  4.472863e-08  4.472863e-08  0.000090  4.477336e-05
UH   1.030439e-05  1.030439e-05  1.030439e-05  0.061837  3.092348e-02


In [15]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    num_tags = len(tag_counts)
    tags = sorted(tag_counts.keys())
    word_count = len(vocab)
    B = np.zeros((num_tags, word_count))

    for i in range(num_tags):
        for j in range(word_count):
            counter = 0
            k =  (tags[i],vocab[j])

            if k in emission_counts:
                counter = emission_counts[k]
                
            tagcounts = tag_counts[k[0]]
            B[i,j] = (counter + alpha)/(tagcounts + alpha * word_count)
    
    return B

In [16]:
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab))

print(f"View Matrix position at row 0, column 0: {B[0,0]:.9f}")
print(f"View Matrix position at row 3, column 1: {B[3,1]:.9f}")

cidx  = ['725','adroitly','engineers', 'promoted', 'synergy']

cols = [vocab[a] for a in cidx]

rvals =['CD','NN','NNS', 'VB','RB','RP']

rows = [states.index(a) for a in rvals]

B_sub = pd.DataFrame(B[np.ix_(rows,cols)], index=rvals, columns = cidx )
print(B_sub)

View Matrix position at row 0, column 0: 0.000006032
View Matrix position at row 3, column 1: 0.000000720
              725      adroitly     engineers      promoted       synergy
CD   8.201296e-05  2.732854e-08  2.732854e-08  2.732854e-08  2.732854e-08
NN   7.521128e-09  7.521128e-09  7.521128e-09  7.521128e-09  2.257091e-05
NNS  1.670013e-08  1.670013e-08  4.676203e-04  1.670013e-08  1.670013e-08
VB   3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08
RB   3.226454e-08  6.456135e-05  3.226454e-08  3.226454e-08  3.226454e-08
RP   3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07


In [17]:
##viterbi algorithm
def initialize(states, tag_counts, A, B, corpus, vocab):

    num_tags = len(tag_counts)
    best_probs = np.zeros((num_tags, len(corpus)))
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)
    
    start_index = states.index("--s--")
    
    for i in range(num_tags):
        
        if A[start_index,i]==0:
            best_probs[i,0] = float('-inf')

        else:
            best_probs[i,0] = math.log(A[start_index,i]) + math.log(B[i,vocab[corpus[0]]])
    
    return best_probs, best_paths

In [18]:
best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)

In [19]:
def viterbi_forward(A, B, test_corpus, best_probs, best_paths, vocab):
    num_tags = best_probs.shape[0]

    for i in range(1, len(test_corpus)): 
        if i % 5000 == 0:
            print("Words processed: {:>8}".format(i))

        for j in range(num_tags):
            p = float('-inf')
            q = None

            for k in range(num_tags):
                probability = best_probs[k,i-1] + math.log(A[k,j])+math.log(B[j,vocab[test_corpus[i]]])
                if probability > p:
                    p = probability
                    q = k

            best_probs[j,i] = p
            best_paths[j,i] = q
    
    return best_probs, best_paths

In [20]:
best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, vocab)

Words processed:     5000
Words processed:    10000
Words processed:    15000
Words processed:    20000
Words processed:    25000
Words processed:    30000


In [21]:
def viterbi_backward(best_probs, best_paths, corpus, states):

    p = best_paths.shape[1] 
    q = [None] * p

    num_tags = best_probs.shape[0]
    
    last_word_best_prob = float('-inf')
    
    output = [None] * p
    
    for i in range(num_tags):

        if best_probs[i,-1]>last_word_best_prob:

            last_word_best_prob = best_probs[i,-1]
    
            q[p - 1] = i
            
    output[p - 1] = states[q[p - 1]]

    for i in range(p-1, 0, -1):
        postag_i = q[i]
        
        q[i - 1] = best_paths[postag_i,i]
        output[i - 1] = states[q[i - 1]]
    
    return output

In [22]:
pred = viterbi_backward(best_probs, best_paths, prep, states)
m=len(pred)
print('The prediction for pred[-7:m-1] is: \n', prep[-7:m-1], "\n", pred[-7:m-1], "\n")
print('The prediction for pred[0:8] is: \n', pred[0:7], "\n", prep[0:7])

The prediction for pred[-7:m-1] is: 
 ['see', 'them', 'here', 'with', 'us', '.'] 
 ['VB', 'PRP', 'RB', 'IN', 'PRP', '.'] 

The prediction for pred[0:8] is: 
 ['DT', 'NN', 'POS', 'NN', 'MD', 'VB', 'VBN'] 
 ['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken']


In [23]:
print('The third word is:', prep[3])
print('Your prediction is:', pred[3])
print('Your corresponding label y is: ', y[3])

The third word is: temperature
Your prediction is: NN
Your corresponding label y is:  temperature	NN



In [24]:
def compute_accuracy(pred, y):
    num_correct = 0
    total = 0
    
    for prediction, y in zip(pred, y):
        word_tag_tuple = y.split()
        
        if len(word_tag_tuple)!=2:
            continue 

        word, tag = word_tag_tuple

        if prediction == tag:

            num_correct += 1

        total += 1

    return num_correct/total

In [25]:
print(f"Accuracy of the Viterbi algorithm is {compute_accuracy(pred, y):.4f}")

Accuracy of the Viterbi algorithm is 0.9531
